# 패키지

In [1]:
import tensorflow as tf
from tensorflow import keras

# GPU 사용 설정

In [2]:
# 사용 가능한 모든 GPU 리스트
gpus = tf.config.list_physical_devices('GPU')

# 사용할 프로세서 선택 : GPU
if gpus:
    try: tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e: print(e)

# 데이터셋

In [3]:
import os

# 데이터셋 주소
dataset_dir = os.path.join('.', 'datasets', 'MusicalSymbol-v.1.1.5')
csv_dir = os.path.join(dataset_dir, 'label.csv')

In [4]:
import pandas as pd

# csv 파일 불러오기
df = pd.read_csv(csv_dir)

# name을 정수값으로 추출
df['name_int'] = df['name'].str.extract('(\d+)').astype(int)

# 정수값을 기준으로 정렬 및 인덱스 초기화
df = df.sort_values('name_int')
df.reset_index(drop=True, inplace=True)

# 필요 없는 name, name_int 열 제거
df = df.drop(columns=['name', 'name_int'])

# pandas 데이터 프레임을 tensorflow 텐서로 변환
df = tf.convert_to_tensor(df.values, dtype=tf.int32)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [5]:
def make_data(img_path):
    # 타겟 생성
    image = tf.io.read_file(img_path)               # 파일 로드
    image = tf.image.decode_png(image, channels=1)  # png 파일로 변환
    image = tf.cast(image, tf.float32)              # uint8 -> float32
    image = image / 255.0                           # 0~1 로 정규화
    image = 1.0 - image                             # 흑백 이미지 반전

    # 레이블 생성
    lable_index = tf.strings.split(img_path, os.path.sep)[-1]           # 파일 이름 추출
    lable_index = tf.strings.split(lable_index, '.')[0]                 # 확장자 제거
    lable_index = tf.strings.to_number(lable_index, out_type=tf.int32)  # 숫자 변환
    lable = tf.gather(df, lable_index-1)                                # df 에서 lable_index 번째 label 추출

    # 타겟과 레이블 반환
    return (image, lable)

In [6]:
# 데이터셋 생성
dataset = tf.data.Dataset.list_files(os.path.join(dataset_dir, '*.png'))

# map
dataset = dataset.map(make_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# repeat
dataset = dataset.repeat(3)

# shuffle
dataset = dataset.shuffle(buffer_size=25000)

# train, validation
ds_validation = dataset.take(320).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_train = dataset.skip(320).batch(32)

# 모델 학습

In [ ]:
# 그래프 초기화
keras.backend.clear_session()

# 모델 생성 및 학습
with tf.device('/device:GPU:0'):
    # 모델 생성
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(192, 512, 1)))
    model.add(tf.keras.layers.MaxPool2D(2, 2))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(192, 512, 1)))
    model.add(tf.keras.layers.MaxPool2D(2, 2))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(192, 512, 1)))
    model.add(tf.keras.layers.MaxPool2D(2, 2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(133, activation='sigmoid'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # 모델 학습
    model.fit(ds_train, epochs=10, validation_data=ds_validation)